In [1]:
from datascience import *
from prob140 import *

# We could also do import prob140.pykov as pk
# and call pk.Vector, etc

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Outline

We don't have very much documentation right now; this notebook is about it: I'll add more documentation later this weekend

`Table().values(...).transition_function(transitionFunction)` is a new syntax where `transitionFunction` computes `P(Y | X)` (in this case, our step probabilities)

In [2]:
def identity_transition(x,y):
    if x==y:
        return 1
    return 0

transMatrix = Table().values('Source',np.arange(1,4),'Target',np.arange(1,4)).transition_function(identity_transition)
transMatrix

C:\ProgramData\Anaconda3\lib\site-packages\datascience\tables.py:362: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\datascience\tables.py:234: FutureWarning: column_labels is deprecated; use labels
  warnings.warn("column_labels is deprecated; use labels", FutureWarning)


Source,Target,P(Target | Source)
1,1,1
1,2,0
1,3,0
2,1,0
2,2,1
2,3,0
3,1,0
3,2,0
3,3,1


In [3]:
def transitionFunction(x,y):
    """
    P(Y | X) = \frac{1}{2} if Y = X or X+1 else 0
    """
    x,y= ord(x),ord(y) # This is just hacky to use letters instead of numbers
    if x == y:
        return .5
    if (x + 1)%5 == y%5:
        return .5
    return 0
    
transMatrix = Table().values('Source',['A','B','C','D','E'],\
                             'Target',['A','B','C','D','E']).transition_function(transitionFunction)

C:\ProgramData\Anaconda3\lib\site-packages\datascience\tables.py:362: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\datascience\tables.py:234: FutureWarning: column_labels is deprecated; use labels
  warnings.warn("column_labels is deprecated; use labels", FutureWarning)


In [4]:
transMatrix

Source,Target,P(Target | Source)
A,A,0.5
A,B,0.5
A,C,0
A,D,0
A,E,0
B,A,0
B,B,0.5
B,C,0.5
B,D,0
B,E,0


## toMarkovChain()

Say you have a table constructed with the above syntax; we use `table.toMarkovChain()` to convert to our custom MarkovChain object; this object lies between the Pykov chain and the Table library

In [5]:
mc =transMatrix.toMarkovChain()
mc

,T: A,T: B,T: C,T: D,T: E
S: A,0.5,0.5,0.0,0.0,0.0
S: B,0.0,0.5,0.5,0.0,0.0
S: C,0.0,0.0,0.5,0.5,0.0
S: D,0.0,0.0,0.0,0.5,0.5
S: E,0.5,0.0,0.0,0.0,0.5


## markovChain.mean_first_passage_time_to

`markovChain.mean_first_passage_time_to(state)`

Computes the mean hitting time starting at any state and ending at specified state `state`

In [6]:
mc.mean_first_passage_time_to('A')

State,Mean Time
B,8
C,6
D,4
E,2


In [7]:
startVector = Table().values(['A','B','C','D','E']).probability([.1,.4,.2,.1,.2])
startVector

Value,Probability
A,0.1
B,0.4
C,0.2
D,0.1
E,0.2


## markovChain.distribution_after_time

`markovChain.distribution_after_time(initial_distribution,steps=1)`

Returns the probability distribution after `steps` timesteps. If $A$ is our transition matrix, then this computes $(\text{initial_distribution})^TA^{\text{steps}}$

The initial distribution is a probability distribution as in prob140; you can also pass in a dictionary too (original pykov)

In [8]:
mc.distribution_after_time(startVector,2)

State,Probability
A,0.15
B,0.2
C,0.275
D,0.225
E,0.15


## markovChain.steady_state

`markovChain.steady_state`

Assuming that the chain is ergodic, this computes the steady state distribution of the chain

In [12]:
mc.steady_state()

State,Probability
A,0.2
B,0.2
C,0.2
D,0.2
E,0.2


## markovChain.accessibility_matrix()

`markovChain.accessibility_matrix()`

Returns a matrix (currently a pandas dataframe, but we can make it a datascience Table too) where $A_{ij} = \begin{cases} 1 & \text{J reachable from I} \\ 0 & \text{J not reachable from I} \end{cases}$

In [9]:
mc.accessibility_matrix() # everything can reach everything

,T: A,T: B,T: C,T: D,T: E
S: A,1,1,1,1,1
S: B,1,1,1,1,1
S: C,1,1,1,1,1
S: D,1,1,1,1,1
S: E,1,1,1,1,1


## markovChain.random_walk

`markovChain.random_walk(length_of_walk,start=None)`

Performs a random walk on the chain of size `length_of_walk`, starting at a state `start` if passed in

In [10]:
random_walk = mc.random_walk(10,start='A') #good for simulations
random_walk

['A', 'A', 'A', 'A', 'A', 'A', 'B', 'C', 'C', 'C', 'D']

## markovChain.probability_of_walk

`markovChain.probability_of_walk(list)`

Takes in a list (which is a walk on the chain) - such a chain can be generated by `random_walk` above;
Returns the probability of getting the walk if doing a random walk

In [11]:
mc.probability_of_walk(['A','A','A','B','C'])

0.06250000000000001